In [1]:
from config import setup
setup()

 # Projet AP GeoSafe, Evacuation lors de feu
 
 ## Generator 
 Pour générer les données utilisées après par le projet, il faut utiliser le générateur du projet evacsim

In [2]:
from docplex.cp.model import CpoModel
from docplex.cp.model import *
import numpy as np

configuration

In [3]:
from config_duc import setup
setup()

## modélisation

In [4]:
def geosafe_model(l, q, eps, r, W, H, d):
    '''
    Int  Matrix l: length path from i to j = G.l[i,j].
                        no path ij <--> G.l[i,j] = 0
    Int  Array  q: capacity of node i <--> G.q[i]
    Bool Array  eps: evacuation node 
    Bool Array  r: safe node
    
    Int  Array  W: initial population at node i <--> W[i]
    Int  H : Time span
    '''
    
    nb_node = q.shape[0]
    total_population = np.sum(W)
    
    G.t = not G.eps and not G.r # transition node

    mdl = CpoModel(name='geosafe')
    
    # == Output ==
    # starting date
    s = np.array( mdl.integer_var_list(nb_node, "s") )
    
    # evacuation rate aka. height of package
    h = np.array( mdl.integer_var_list(nb_node, "h") )

    # == Intermediate ==
    # node flow of population
    phi = np.matrix([
            [ 
                mdl.integer_var(name='phi[%d,%d]'%(i,j)) for j in range(H) 
            ] for i in range(nb_node)
        ])
    
    # ending date (leaving time) of node
    e = np.array( mdl.integer_var_list(nb_node, "e") )
    
    mdl.add_constraint(
        e == s + W/h
    )
    
    # == Constraints ==
    # evacuate everyone in evacuation node
    mdl.add_constraint(
        np.sum(phi, axis=0) == W
    ) 
    
    # Flow at a node u = sum from all of its leaves node epsilon
#     mdl.add_constraint(
#         phi[u, t] == np.sum( phi[u, t - l[u,v] - s[eps]] ) \
#             for t in range(H)                                  \
#             for v in np.where(eps)                           \
#             for u in np.where(t)                             
#     )
    # simplified version
    mdl.add_constraint(
        phi[u, t] == np.sum(h[v]) \
                for t in range[s[v] + l[u, v], e[v] + l[u, v]]
    )
    
    # Flow at a node does not excess capacity of arc
    mdl.add_constraint(
        phi[:,t] <= q for t in range [H]
    )
    
    # Objective
    mdl.add(
        minimize(np.max(s[eps] + phi[eps]/W[eps] - d[eps]))
    )
    

## Unit test:

In [7]:
import networkx as nx

In [14]:
G = nx.Graph()

nodes_state = [
    (1, {'eps': True, 'r': False}),
    (2, {'eps': True, 'r': False}),
    (3, {'eps': True, 'r': False}),
    (4, {'eps': False, 'r': False}),
    (5, {'eps': False, 'r': False}),
    (6, {'eps': False, 'r': True})
]

edges_state = [
    (1, 4, {'l' : 4}),
    (2, 4, {'l' : 3}),
    (4, 5, {'l' : 7}),
    (3, 5, {'l' : 3}),
    (5, 6, {'l' : 10})
]

G.add_nodes_from(nodes_state)
G.add_edges_from(edges_state)

AttributeError: 'dict' object has no attribute 'itervalues'

In [64]:
a = nx.get_node_attributes(G, 'eps').values()
for item in a:
    eps += item
print(eps)

[3. 3. 3. 3. 3. 3.]


In [ ]:
l = nx.to_numpy_matrix(G, weight='l')
geosafe_model(l, q, eps, r, W, H, d)